In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')

In [3]:
df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [4]:
print(df['NObeyesdad'].unique())

print(df.shape)

['Overweight_Level_II' 'Normal_Weight' 'Insufficient_Weight'
 'Obesity_Type_III' 'Obesity_Type_II' 'Overweight_Level_I'
 'Obesity_Type_I']
(20758, 18)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [6]:
df.isnull().values.any()

False

In [7]:
factor, label= pd.factorize(df['NObeyesdad'])
df.NObeyesdad = factor

In [8]:
gender = pd.factorize(df['Gender'])
df.Gender = gender[0]

family_history_with_overweight = pd.factorize(df['family_history_with_overweight'])
df.family_history_with_overweight = family_history_with_overweight[0]

FAVC = pd.factorize(df['FAVC'])
df.FAVC = FAVC[0]

In [9]:
CAEC = pd.factorize(df['CAEC'])
df.CAEC = CAEC[0]

SMOKE = pd.factorize(df['SMOKE'])
df.SMOKE = SMOKE[0]

SCC = pd.factorize(df['SCC'])
df.SCC = SCC[0]

CALC = pd.factorize(df['CALC'])
df.CALC = CALC[0]

MTRANS = pd.factorize(df['MTRANS'])
df.MTRANS = MTRANS[0]

In [10]:
df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,0,24.443011,1.699998,81.669950,0,0,2.000000,2.983297,0,0,2.763573,0,0.000000,0.976473,0,0,0
1,1,1,18.000000,1.560000,57.000000,0,0,2.000000,3.000000,1,0,2.000000,0,1.000000,1.000000,1,1,1
2,2,1,18.000000,1.711460,50.165754,0,0,1.880534,1.411685,0,0,1.910378,0,0.866045,1.673584,1,0,2
3,3,1,20.952737,1.710730,131.274851,0,0,3.000000,3.000000,0,0,1.674061,0,1.467863,0.780199,0,0,3
4,4,0,31.641081,1.914186,93.798055,0,0,2.679664,1.971472,0,0,1.979848,0,1.967973,0.931721,0,0,0


In [11]:
x = df[df.columns[1:17]]
y = df['NObeyesdad']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 21)

In [13]:
classifier = RandomForestClassifier(n_estimators = 15, min_samples_leaf = 5, criterion = 'gini', random_state = 40)
classifier.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=5, n_estimators=15, random_state=40)

In [14]:
y_pred = classifier.predict(X_test)

In [15]:
pd.crosstab(y_test, y_pred,rownames=['Actual'], colnames=['Predicted'])

Predicted,0,1,2,3,4,5,6
Actual,,,,,,,
0,412,16,0,0,0,38,25
1,8,567,41,0,0,15,0
2,0,35,484,0,0,0,0
3,0,0,1,817,1,0,0
4,2,0,0,0,586,0,19
5,69,57,1,0,0,347,12
6,40,4,1,3,20,13,518


In [16]:
test_df = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')

In [17]:
test_df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [18]:
gender = pd.factorize(test_df['Gender'])
test_df.Gender = gender[0]

family_history_with_overweight = pd.factorize(test_df['family_history_with_overweight'])
test_df.family_history_with_overweight = family_history_with_overweight[0]

FAVC = pd.factorize(test_df['FAVC'])
test_df.FAVC = FAVC[0]

CAEC = pd.factorize(test_df['CAEC'])
test_df.CAEC = CAEC[0]

SMOKE = pd.factorize(test_df['SMOKE'])
test_df.SMOKE = SMOKE[0]

SCC = pd.factorize(test_df['SCC'])
test_df.SCC = SCC[0]

CALC = pd.factorize(test_df['CALC'])
test_df.CALC = CALC[0]

MTRANS = pd.factorize(test_df['MTRANS'])
test_df.MTRANS = MTRANS[0]

In [19]:
test_pred = classifier.predict(test_df[test_df.columns[1:]])

In [20]:
test_pred

array([4, 5, 3, ..., 2, 1, 4])

In [21]:
test_predictions_labels = label[test_pred]

In [22]:
submission_df = pd.DataFrame({'id': test_df.id, 'NObeyesdad': test_predictions_labels})

In [23]:
submission_df

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Normal_Weight
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [24]:
# submission.to_csv('submission_df')